# Week 2-3: Logistic Regression, Differential Privacy Basics


We will consider the task of binary classification where we are given labeled data $\{(x_i , y_i )\}_{i \in[n]}$
with $x_i \in \mathbb{R}^d$ and $y_i \in {−1, 1}$, and the goal is to learn a classifier $f_\theta : \mathbb{R}^d \rightarrow \{−1, 1\}$ parameterized by $\theta \in \mathbb{R}^d$ . Given a new sample $x$, our predicted label would be $\hat{y} = f_\theta (x)$. One
way to learn such a classifier is by using the logistic model defined using the conditional
probabilty


$P(y = 1|x, θ) = \frac{e^{\theta^T x}}{1+ e^{\theta^T x}}$

a) For the above model, verify that checking P(y = 1|x) > P(y = −1|x) is equivalent
to checking θ >x > 0. Given this, what would be your strategy to estimate the label
once you know θ?

**Solution:**


$$\begin{align*}
P(y = -1|x, θ) &=  1 - P(y = 1|x, θ) \\
&= 1 -\frac{e^{\theta^T x}}{1+ e^{\theta^T x}} \\ 
&= \frac{1}{1+ e^{\theta^T x}}
\end{align*}$$

Then 
$$P(y = 1|x, θ) > P(y = -1|x, θ)$$
$$\frac{e^{\theta^T x}}{1+ e^{\theta^T x}} > \frac{1}{1+ e^{\theta^T x}}$$
$$e^{\theta^T x} > 1$$
$$\theta^T x >\ln{1}$$
$$\theta^T x > 0$$


This gives us a the intuition that for given sample $x$ we might assign a label $y=1$ if $\theta^T x > 0$ and $y=-1$ if $\theta^T x < 0$


b) Write down the log likelihood function for the above model and describe an algorithm
to find the maximum likelihood estimate for θ.

**Solution**

The maximum likelihood function is
$$\begin{align*}
\mathcal{L}(\theta) &= \prod_{y_i} P(y=1 |x,\theta) \cdot \prod_{y_i} (1 - P(y=1 |x,\theta)) \\
&= \prod P(y=1 |x,\theta)^{y_i} (1 - P(y=1 |x,\theta))^{(1-y_i)}
\end{align*}
$$

Finding the log-likelihood, we get:
$$
\begin{align*}
\mathcal{l}(\theta) &= \sum y_i \log{P(y=1 |x,\theta)} + (1-y_i) \log{1 - P(y=1 |x,\theta)} \\
&= \sum y_i \log{\frac{e^{\theta^T x}}{1+ e^{\theta^T x}}} + (1-y_i) \log{\frac{1}{1+e^{\theta^T x}}} \\ 
& = \sum y_i [\log{\frac{e^{\theta^T x}}{1+e^{\theta^T x}}} - \log{\frac{1}{1+e^{\theta^T x}}}] + \log{\frac{1}{1+e^{\theta^T x}}} \\
&= \sum y_i [\log{(e^{\theta^T x})} - \log{(1+e^{\theta^T x})} + \log{(1+e^{\theta^T x}}] + \log{\frac{1}{1+e^{\theta^T x}}} \\
&= \sum y_i \log{(e^{\theta^T x})} - \log{(1+e^{\theta^T x})} \\ 
&= \sum y_i \theta^T x - \log{(1+e^{\theta^T x})} 
\end{align*}$$
For maximizing the log-likelihood function we take the derivative and find a zero:
$$\begin{align*}
\nabla_{\theta} \mathcal{l}(\theta) &= \nabla_{\theta}[\sum y_i \theta^T x_i - \log{(1+e^{\theta^T x_i})}] \\ 
&= \sum y_i x_i - \frac{ e^{\theta^T x}}{1+e^{\theta^T x}} x_i 
\end{align*}
$$

We can find a root using gradient descent method.

In [4]:
import numpy as np

In [25]:
def log_likelihood(theta, X, y):    
    
    linear_combination = np.dot(X, theta)
      
    log_ll = np.sum(y * linear_combination - np.log(1 + np.exp(linear_combination)))    
    
    return log_ll

In [28]:
def log_likelihood_derivative(theta, X, y):

    n = len(y)
    gradient = np.zeros_like(theta)
    for i in range(n):
        linear_combination = np.dot(X[i], theta)
        gradient += (y[i]*X[i] - (( np.exp(linear_combination)/(1+ np.exp(linear_combination)))* X[i]))
    return gradient

    return derivative

In [29]:
n = 1
d = 50
X = np.random.normal(loc=0, scale=np.sqrt(0.1), size=(n,d))

theta_star = np.random.rand(d)

linear_combination = X.dot(theta_star)

probabilities = 1 / (1 + np.exp(-linear_combination))

y = np.array((np.where(probabilities >= 0.5, 1, -1)))

In [30]:
linear_combination

array([0.84531268])

In [35]:
def gradient_descent(f, gradient, X, y, theta, learning_rate=0.001, tol=1e-6, max_iter=1000):
    for _ in range(max_iter):
        grad = gradient(theta, X, y)
        theta -= learning_rate * grad
        if np.linalg.norm(grad) < tol:
            return theta
    raise ValueError("Gradient descent did not converge within the maximum number of iterations.")

In [34]:

theta_initial_guess = theta_star

theta_mle = newton_raphson(log_likelihood, log_likelihood_derivative, X, y, theta_initial_guess)
theta_mle

[  -9.3081527    -9.58185617  -10.19953727    2.61882867   43.41229769
  -12.05154171  -16.66335118    2.98531392   18.72890907  -53.72256107
   10.15145218    8.33649757   16.36283494    2.74419291   -5.81082798
 -166.53131132  139.28017556  -83.58350133   -5.17829235   -4.1704819
   31.94894149    4.08714374    2.82237736 -118.03617313   14.08977089
    7.85986817    4.23795541    2.82820154   13.51261653    4.63552332
  -29.04547934  -14.67906798    4.69306428   -8.57629674   13.33028082
   14.52128258   -1.51251242   69.80780963  -12.41323644    2.1487699
    4.12697672   -1.80082259   -7.83456398   -5.13775123   27.43616668
  -26.69596401  138.02530491    2.41536592   -6.17694067   20.84643039]
[ -19.05289604  -19.31416025  -21.24094647    5.14203494   86.1311065
  -24.16929188  -33.7333884     5.50077411   36.70241835 -107.51589944
   19.8964893    16.17620639   32.13553962    5.13160483  -11.66833549
 -334.01550969  278.42924688 -167.22456843  -11.23025254   -9.30398641
   62.96

ValueError: Newton-Raphson method did not converge within the maximum number of iterations.

In [37]:
theta_initial = np.random.rand(d)  # Initial guess for theta

# Run gradient descent
theta_estimate = gradient_descent(log_likelihood, log_likelihood_derivative, X, y, theta_initial)

print("Estimated theta:", theta_estimate)


ValueError: Gradient descent did not converge within the maximum number of iterations.

## 2. Differential Privacy Basics

### a)
- As $\epsilon$ tends to $0$, the limits of the functions $\frac{e^x}{1 + e^x}$ and $\frac{1}{1 + e^x}$ as $x$ approaches $0$ are both equal to $\frac{1}{2}$. This option leaves $y_i$ equal to $x_i$ with a probability of one half and equal to $1-x_i$ with a probability of one half. In this case, the information will remain private until the point of information loss as well.
- As $\epsilon$ tends to $\infty$, the limits of the functions $\frac{e^x}{1 + e^x}$ and $\frac{1}{1 + e^x}$ as $x$ approaches $0$ are $1$ and $0$ respectively. This scenario clearly indicates that user information becomes blatantly non-private.
- Let us use the expected value as an estimator for $y_i$, then
  $$\begin{align*}
  E[y_i] &= x_i (\frac{e^\epsilon}{1+e^\epsilon}) + (1-x_i)(\frac{1}{1+e^\epsilon}) \\
  &= \frac{x_i e^\epsilon}{1+e^\epsilon} + \frac{1-x_i}{1+e^\epsilon} \\
  &= \frac{x_i e^\epsilon + 1 - x_i}{1+e^\epsilon} \\
  &= \frac{e^\epsilon + x_i(1-e^\epsilon)}{1+e^\epsilon}
  \end{align*}$$
  